## Step 1 - Start local geth node


In [200]:
# Step 1 - Start local geth node
# Run runGeth script    
# $ ./00_runGeth.sh
# 
# Or the following but remember to include the preloaded keyfiles for all actors 
# geth --dev --dev.period 1 --datadir ./testchain --rpc --rpccorsdomain '*' --rpcport 8646 --rpcapi "eth,net,web3,debug" --port 32323 --maxpeers 0 --targetgaslimit 994712388 console

In [52]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import web3, solc, json
from web3 import Web3
from web3.auto.gethdev import w3

In [53]:
# ipc:./testchain/geth.ipc

w3 = Web3(Web3.IPCProvider("./testchain/geth.ipc"))

In [54]:
from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)

In [55]:
# Set Up Environment

In [64]:
# Check that our geth node is running
block = w3.eth.getBlock('latest')
block.hash

HexBytes('0xd09cc60ef10885774f6532a6e8cec055da89a9ad20851d34032a860bccac2510')

In [67]:
# that our test accounts have loaded  
w3.eth.accounts[3]
# = '0xa33a6c312D9aD0E0F2E95541BeED0Cc081621fd0'

'0xa33a6c312D9aD0E0F2E95541BeED0Cc081621fd0'

## Step 2 - Set Up Environment

In [68]:
# compile contract, check which version. ie solidity 0.4.25
# !pip install solc
!solc --version

solc, the solidity compiler commandline interface
Version: 0.4.25+commit.59dbf8f1.Darwin.appleclang


In [235]:
# load web3 python helper functions 
import functions

In [208]:
# Unlock user accounts
functions.unlockAccounts('');

In [209]:
# Test helper fucntions 
functions.printBalances()

 # Account                                      EtherBalanceChange                   Token Name
-- ------------------------------------------ -------------------- ----------------------- ---------------------
 0 0xa00Af22D07c87d96EeeB0Ed583f8F6AC7812827E                    0                       0 Account #0 - Miner
 1 0xa11AAE29840fBb5c86E6fd4cF809EBA183AEf433                    0                       0 Account #1 - Owner
 2 0xa22AB8A9D641CE77e06D98b7D7065d324D3d6976                    0                       0 Account #2 - Alice
 3 0xa33a6c312D9aD0E0F2E95541BeED0Cc081621fd0                    0                       0 Account #3 - Bob
 4 0xa44a08d3F6933c69212114bb66E2Df1813651844                    0                       0 Account #4 - Carol
 5 0xa55A151Eb00fded1634D27D1127b4bE4627079EA                    0                       0 Account #5 - Dave
 6 0xa66a85ede0CBE03694AA9d9dE0BB19c99ff55bD9                    0                       0 Account #6
 7 0xa77A2b9D4B1c010A22A7c565Dc4

In [71]:
# Move files to testing directory 

SOURCEDIR="../../contracts"
REDBLACKLIBSOL="BokkyPooBahsRedBlackTreeLibrary.sol"
TESTREDBLACKSOL="TestBokkyPooBahsRedBlackTree.sol"
REDBLACKLIBDIR = SOURCEDIR + REDBLACKLIBSOL

In [72]:
!cp $SOURCEDIR/$REDBLACKLIBSOL . 
!cp $SOURCEDIR/$TESTREDBLACKSOL .

## Step 3 - Compile using solc (Option 1)

In [78]:
RedBlackTreeLibraryOutput = !solc --optimize --combined-json abi,bin,interface {REDBLACKLIBSOL}

In [79]:
redBlackTreeLibraryAbi = json.loads(RedBlackTreeLibraryOutput[0])['contracts'][REDBLACKLIBSOL + ':BokkyPooBahsRedBlackTreeLibrary']['abi'] 
redBlackTreeLibraryAbi

'[{"anonymous":false,"inputs":[{"indexed":false,"name":"where","type":"string"},{"indexed":false,"name":"action","type":"string"},{"indexed":false,"name":"key","type":"uint256"},{"indexed":false,"name":"parent","type":"uint256"},{"indexed":false,"name":"left","type":"uint256"},{"indexed":false,"name":"right","type":"uint256"},{"indexed":false,"name":"red","type":"bool"}],"name":"Log","type":"event"}]'

In [77]:
redBlackTreeLibraryBin = json.loads(RedBlackTreeLibraryOutput[0])['contracts'][REDBLACKLIBSOL + ':BokkyPooBahsRedBlackTreeLibrary']['bin']
redBlackTreeLibraryBin

'604c602c600b82828239805160001a60731460008114601c57601e565bfe5b5030600052607381538281f30073000000000000000000000000000000000000000030146080604052600080fd00a165627a7a72305820268cf9bad0cd48fa6d3885e00ffdc18d5d4538e6f47cb32165d77cb426a8cd7e0029'

In [ ]:
# Build the contract object in preparation for deployment
token_deployer = w3.eth.contract(abi=redBlackTreeLibraryAbi, bytecode=redBlackTreeLibraryBin)

## Step 3 - Compile using Remix (Option 2)

In [80]:
#Copied bytecode from Remix 

bytecode = {
	"linkReferences": {},
	"object": "604c602c600b82828239805160001a60731460008114601c57601e565bfe5b5030600052607381538281f30073000000000000000000000000000000000000000030146080604052600080fd00a165627a7a72305820c716422ec8835c7100eaa0d9282eccfbcac668f51b1912146db73d8f872ef2bc0029",
	"opcodes": "PUSH1 0x4C PUSH1 0x2C PUSH1 0xB DUP3 DUP3 DUP3 CODECOPY DUP1 MLOAD PUSH1 0x0 BYTE PUSH1 0x73 EQ PUSH1 0x0 DUP2 EQ PUSH1 0x1C JUMPI PUSH1 0x1E JUMP JUMPDEST INVALID JUMPDEST POP ADDRESS PUSH1 0x0 MSTORE PUSH1 0x73 DUP2 MSTORE8 DUP3 DUP2 RETURN STOP PUSH20 0x0 ADDRESS EQ PUSH1 0x80 PUSH1 0x40 MSTORE PUSH1 0x0 DUP1 REVERT STOP LOG1 PUSH6 0x627A7A723058 KECCAK256 0xc7 AND TIMESTAMP 0x2e 0xc8 DUP4 0x5c PUSH18 0xEAA0D9282ECCFBCAC668F51B1912146DB7 RETURNDATASIZE DUP16 DUP8 0x2e CALLCODE 0xbc STOP 0x29 ",
	"sourceMap": "637:14195:0:-;;132:2:-1;166:7;155:9;146:7;137:37;252:7;246:14;243:1;238:23;232:4;229:33;270:1;265:20;;;;222:63;;265:20;274:9;222:63;;298:9;295:1;288:20;328:4;319:7;311:22;352:7;343;336:24"
}


In [81]:
token_code = bytecode['object']

In [82]:
token_code

'604c602c600b82828239805160001a60731460008114601c57601e565bfe5b5030600052607381538281f30073000000000000000000000000000000000000000030146080604052600080fd00a165627a7a72305820c716422ec8835c7100eaa0d9282eccfbcac668f51b1912146db73d8f872ef2bc0029'

In [83]:
#Copied ABI from Remix 

token_abi = '''

[
	{
		"anonymous": false,
		"inputs": [
			{
				"indexed": false,
				"name": "where",
				"type": "string"
			},
			{
				"indexed": false,
				"name": "action",
				"type": "string"
			},
			{
				"indexed": false,
				"name": "key",
				"type": "uint256"
			},
			{
				"indexed": false,
				"name": "parent",
				"type": "uint256"
			},
			{
				"indexed": false,
				"name": "left",
				"type": "uint256"
			},
			{
				"indexed": false,
				"name": "right",
				"type": "uint256"
			},
			{
				"indexed": false,
				"name": "red",
				"type": "bool"
			}
		],
		"name": "Log",
		"type": "event"
	}
]
'''.strip()

In [84]:
# Build the contract object in preparation for deployment
token_deployer = w3.eth.contract(abi=token_abi, bytecode=token_code)


## Step 4 - Build transaction


In [87]:
init = token_deployer.constructor(1000)
txn = init.buildTransaction({'gas': 320000})

In [88]:
# define
def sign_transaction(txn):
    # Flesh out the transaction for local signing
    next_nonce = w3.eth.getTransactionCount(str(w3.eth.accounts[1]))
    signable_transaction = dict(
      txn,
      nonce=next_nonce,
      gasPrice=w3.toWei(4, 'gwei'),
    )
    # Sign transaction
    signature_info = w3.eth.account.signTransaction(signable_transaction)
    # Broadcast transaction
    txn_hash = w3.eth.sendRawTransaction(signature_info.rawTransaction)
    # Wait for the transaction to be mined
    receipt = w3.eth.waitForTransactionReceipt(txn_hash)

    return receipt

In [89]:
# Test Code
receipt = sign_transaction(txn)
receipt

TypeError: signTransaction() missing 1 required positional argument: 'private_key'

In [ ]:
# Get deployed contract
token = w3.eth.contract(address=receipt.contractAddress, abi=token_abi)

In [ ]:
# Check your balance
token.functions.balanceOf(acct.address).call()